In [8]:
import os
import geopandas as gpd
import py3dep
import copy 

hucs_on_carbs_with_sinks = gpd.read_file('huc12_on_carbs_with_sinks.shp')

max_tries = 2
dem_res = 10

rasterdir = 'carb_huc_dems'
if not os.path.exists(rasterdir):
        os.makedirs(rasterdir)


for idx, row in hucs_on_carbs_with_sinks.iterrows():
    if idx<2:
        huc12_str = copy.copy(row.huc12)
        this_hu12 = row.geometry
        rasterfile = huc12_str + "-3DEP.tif"
        
        finished = False
        tries = 0

        full_rasterfile_path = os.path.join(rasterdir, rasterfile)
        hucs_completed = []
        hucs_failed = []

        # Check if we already have the dem
        if not os.path.isfile(full_rasterfile_path):
            while not finished:
                try:
                    print("Downloading dem for ", huc12_str)
                    dem = py3dep.get_dem(this_hu12, dem_res)
                    finished = True
                    failed = False
                    # Convert to projected CRS
                    dem = dem.rio.reproject(5070)
                    dem.rio.to_raster(full_rasterfile_path)
                    hucs_completed.append(huc12_str)
                except Exception as error:
                    print("Failed to retrieve DEM for", huc12_str + ".")
                    print("error:", error)
                    tries += 1
                    if tries > max_tries:
                        finished = True
                        failed = True
                        hucs_failed.append(huc12_str)
            if failed:
                print("Failed to retrieve the DEM ("+ huc12_str +") after", str(max_tries), "tries.")       
        else:
            print("We already have dem raster", rasterfile, " continuing without download.")

completed_file = os.path.join(rasterdir, 'completed_dems.txt')
with open(completed_file, 'w') as f:
    for huc in hucs_completed:
        f.write("%s\n" % huc)

failed_file = os.path.join(rasterdir, 'failed_dems.txt')
with open(failed_file, 'w') as f:
    for huc in hucs_failed:
        f.write("%s\n" % huc)



In [10]:
dem.rio.r

{'3DEPElevation': 'None#_FillValue=nan',
 'AREA_OR_POINT': 'Area',
 'DataType': 'Generic',
 'SourceBandIndex': 0,
 '_FillValue': nan,
 'nodatavals': (nan,)}

In [15]:
idx =  hucs_on_carbs_with_sinks.iloc[0]
idx

objectid                                                  290.0
tnmid                    {2CA97DB1-206D-479F-AD9B-BFFF02D6B080}
metasource               {7E9D2B22-BF65-426E-9174-661A4609B6FA}
sourcedata                                                 None
sourceorig                                                 None
sourcefeat                                                 None
loaddate                                2017/05/25 13:11:26.000
referenceg                                                 None
areaacres                                              11009.96
areasqkm                                                  44.56
states                                                    VA,WV
huc12                                              050500020603
name                                      Headwaters East River
hutype                                                        S
humod                                                        OT
tohuc                                   

In [12]:

hucs_on_carbs_with_sinks[idx]

KeyError: 0